In [163]:
import pandas as pd 
import tensorflow as tf 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import pickle

In [164]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [165]:
# Removing unwanted cols
df_imp_col = df.drop(
    columns=['RowNumber','CustomerId','Surname'], #df.columns[:3],
    axis=1
)

df_imp_col.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [166]:
# Checking for missing values
df_imp_col.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [167]:
# Encoding the categorical features

# Getting the categorical feature names
def cat_cols_func(df):
    res = [i for i in df.columns if df[i].dtype == 'object']
    return res

cat_cols = cat_cols_func(df_imp_col)
cat_cols

['Geography', 'Gender']

In [168]:
# Encoding the binary values cols with label encoder

def binary_cat(df, cat_cols):
    df = df[cat_cols]
    bin_cat_cols = [i for i in cat_cols if df[i].nunique() <= 2]
    return bin_cat_cols

bin_cat_cols = binary_cat(df_imp_col, cat_cols)
bin_cat_cols

['Gender']

In [169]:
# Actually encoding binary categorical cols with label encoder
leo = LabelEncoder()

for i in bin_cat_cols:
    df_imp_col[i] = leo.fit_transform(df_imp_col[i])

df_imp_col.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [170]:
multi_cat_cols = [col for col in cat_cols if col not in bin_cat_cols]
multi_cat_cols

['Geography']

In [171]:
# Encoding multi cardinality cols with ohe encoder

ohe_obj = OneHotEncoder()
encoded_cols = ohe_obj.fit_transform(df_imp_col[['Geography']])
encoded_cols

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [172]:
ohe_obj.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [173]:
encoded_cols_df = pd.DataFrame(encoded_cols.toarray(), columns=ohe_obj.get_feature_names_out(['Geography']))
encoded_cols_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [174]:
# Mergin data frames
df_encoded = pd.concat([df_imp_col.drop('Geography', axis=1), encoded_cols_df], axis=1)
df_encoded

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [175]:
# Generating pickel files
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(leo, file)

with open('ohe_encoder_geography.pkl', 'wb') as file:
    pickle.dump(ohe_obj, file)


In [176]:
# Training and test split
x = df_encoded.drop('Exited', axis=1)
y = df_encoded['Exited']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [177]:
# Scaling all the data

ss_obj = StandardScaler()

x_train = ss_obj.fit_transform(x_train)
x_test = ss_obj.transform(x_test)

In [178]:
# Pickel file for standardscaler

with open('standard_scaler.pkl', 'wb') as file:
    pickle.dump(ss_obj, file)


In [179]:
# Implementing ANN

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping

In [180]:
bank_churn_classifi_model = Sequential(
    [
        # Input and HL1
        Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
        # HL2
        Dense(128, activation='relu'),
        # HL3
        Dense(32, activation='relu'),
        # Output
        Dense(1, activation='sigmoid')
    ]
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [181]:
bank_churn_classifi_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [182]:
# Setting up early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [183]:
# Fitting the model
bank_churn_classifi_model.fit(
    x_train,
    y_train,
    validation_data=(x_test,y_test),
    epochs=500,
    callbacks=early_stop
)

Epoch 1/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8007 - loss: 0.4651 - val_accuracy: 0.8520 - val_loss: 0.3720
Epoch 2/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8531 - loss: 0.3570 - val_accuracy: 0.8545 - val_loss: 0.3588
Epoch 3/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8686 - loss: 0.3215 - val_accuracy: 0.8585 - val_loss: 0.3520
Epoch 4/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8686 - loss: 0.3274 - val_accuracy: 0.8590 - val_loss: 0.3464
Epoch 5/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8701 - loss: 0.3214 - val_accuracy: 0.8555 - val_loss: 0.3546
Epoch 6/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8722 - loss: 0.3213 - val_accuracy: 0.8500 - val_loss: 0.3569
Epoch 7/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8742 - loss: 0.3131 - val_accuracy: 0.8560 - val_loss: 0.3522
Epoch 8/500
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8659 - loss: 0.3186 - val_accu

In [184]:
# saving the model

bank_churn_classifi_model.save('bank_churn_classification.h5')